In [1]:
import os
import re
import glob
import urllib
import toytree
import toyplot
import toyplot.pdf
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from io import StringIO

from Bio import SeqIO
from Bio import Entrez
from Bio import AlignIO
from Bio.Align.Applications import MuscleCommandline

from BCBio import GFF

### Strains that expected to see bai (bile acid-inducible) operon

- *Dorea longicatena*
- *Bacteroides vulgatus*
- *Enterocloster bolteae*
- *Holdemania filiformis*
- *Bacteroides xylanisolvens*
- *Bifidobacterium adolescentis*
- *Clostridium hylemonae*
- *Roseburia intestinalis*
- *Clostridium scinden*

Use the bai operon genes from *clostridium scindens VPI 12708* as reference:
https://www.uniprot.org/uniprot/?query=clostridium%20scindens&fil=reviewed%3Ayes&sort=score

- *baiB*: `P19409`, https://www.uniprot.org/uniprot/P19409
- *baiCD*: `P19410`, https://www.uniprot.org/uniprot/P19410
- *baiE*: `P19412`, https://www.uniprot.org/uniprot/P19412
- *baiA2*: `P19337`, https://www.uniprot.org/uniprot/P19337
- *baiF*: `P19413`, https://www.uniprot.org/uniprot/P19413
- *baiG*: `P32369`, https://www.uniprot.org/uniprot/P32369
- *baiH*: `P32370`, https://www.uniprot.org/uniprot/P32370
- *baiI*: `P32371`, https://www.uniprot.org/uniprot/P32371



In [2]:
os.chdir("/Users/rootqz/Desktop/ReyLab/paper/bile_acid/analysis/hmm/")

acc_bai = ['P19409', 'P19410', 'P19412', 'P19337', 'P19413', 'P32369', 'P32370', 'P32371']

metadata = pd.read_csv("data/metadata.tsv", sep="\t")

In [3]:
# download reference protein sequences from UniProt
records = []
records_dict = {}
for acc in acc_bai:
    handle = urllib.request.urlopen("http://www.uniprot.org/uniprot/{}.xml".format(acc))
    record = SeqIO.read(handle, "uniprot-xml")
    # print("{}, {}".format(acc, record.id))
    SeqIO.write(record, "bai/ref_uniprot/{}.fasta".format(acc), "fasta")
    records_dict[acc] = record
    records.append(record)
    SeqIO.write(records, "bai/bai_clostridium_scindens_UniProt.fasta", "fasta")

In [4]:
# build blast reference
subprocess.call("makeblastdb -in bai/bai_clostridium_scindens_UniProt.fasta -dbtype prot", stdout=True, shell=True)


0

In [6]:
# run blastp
for genome_file in sorted(glob.glob("data/protein_faa/*.faa")):
    # print(genome_file)
    genome_id = genome_file.split("/")[-1][:-12]
    taxa = metadata[metadata["genome_id"] == genome_id]["name"].tolist()[0]

    subprocess.call("blastp -db bai/bai_clostridium_scindens_UniProt.fasta \
                            -query {} \
                            -evalue 1e-5 \
                            -outfmt 6 \
                            -out bai/result/{}.blastp.out".format(genome_file, genome_id), stdout=True, shell=True)
    

In [44]:
# concat blastp results
for genome_file in sorted(glob.glob("data/protein_faa/*.faa")):
    # print(genome_file)
    genome_id = genome_file.split("/")[-1][:-12]
    taxa = metadata[metadata["genome_id"] == genome_id]["name"].tolist()[0]
    
    blastp_out = pd.read_csv("bai/result/{}.blastp.out".format(genome_id), sep="\t", header=None)
    blastp_out.columns = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 
                          'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
    blastp_out['genome_id'] = [genome_id]*blastp_out.shape[0]
    blastp_out['taxa'] = [taxa]*blastp_out.shape[0]
    
    if genome_file == sorted(glob.glob("data/protein_faa/*.faa"))[0]:
        blastp_out_all = pd.DataFrame(columns = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 
                                                 'gapopen', 'qstart', 'qend', 'sstart', 'send', 
                                                 'evalue', 'bitscore',
                                                 'genome_id', 'taxa'])

    blastp_out_all = pd.concat([blastp_out_all, blastp_out], axis=0)
    
blastp_out_all.to_csv("bai/blastp_out_all_genomes.tsv", sep="\t", index=None)

In [71]:
# extract CDS annotation
for genome_file in sorted(glob.glob("data/protein_faa/*.faa")):

    genome_id = genome_file.split("/")[-1][:-12]
    taxa = metadata[metadata["genome_id"] == genome_id]["name"].tolist()[0]
    
    gff = pd.read_csv("data/genome_gff/{}_genomic.gff".format(genome_id), sep="\t", comment='#', header=None)
    gff.columns = ['genome', 'ref_type', 'gene_type', 'start', 'end', 'i1', 'strand', 'i2', 'i3']
    gff_cds = gff[gff['gene_type'] == "CDS"]
    gff_cds['gene'] = [x.split(";")[0].split("-")[1] for x in gff_cds.iloc[:,8].tolist()]
    gff_cds['genome_id'] = [genome_id]*gff_cds.shape[0]
    gff_cds['taxa'] = [taxa]*gff_cds.shape[0]
    this_gff = gff_cds[['genome', 'start', 'end', 'strand', 'gene', 'genome_id', 'taxa']]
    
    if genome_file == sorted(glob.glob("data/protein_faa/*.faa"))[0]:
        gff_out = pd.DataFrame(columns = ['genome', 'start', 'end', 'strand', 'gene', 'genome_id', 'taxa'])

    gff_out = pd.concat([gff_out, this_gff], axis=0)
    
gff_out.to_csv("bai/gff_out_all_genomes.tsv", sep="\t", index=None)

/Users/rootqz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/rootqz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/rootqz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a